# Creating a digital twin
This notebook contains sample [EDSL](https://docs.expectedparrot.com/) code for creating an AI agent and prompting it to critique some content. The code is readily editable to create other agents and survey questions with any available language models. 

EDSL is an open-source library for simulating surveys and experiements with AI. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/docs/notebooks/digital_twin.ipynb)

In [1]:
# ! pip install edsl

In [2]:
from edsl.questions import (
    QuestionMultipleChoice,
    QuestionCheckBox,
    QuestionFreeText,
    QuestionLinearScale,
    QuestionList,
    QuestionBudget,
)
from edsl import Agent, Scenario, Survey, Model

In [3]:
# Construct relevant traits as a dictionary
agent_traits = {
    "persona": """You are a middle-aged mom in Cambridge, Massachusetts. 
        You hope to own a driverless minivan in the near future.
        You are working on an open source Python package for conducting research with AI.""",
    "age": 45,
    "location": "US",
    "industry": "information technology",
    "company": "Expected Parrot",
    "occupation": "startup cofounder",
    "hobbies": "kayaking, beach walks",
}

# Pass the traits and an optional name to an agent
agent = Agent(name="Robin", traits=agent_traits)

In [4]:
# Optionally create some special instructions for the task
context = """You are answering questions about a software package for conducting surveys and experiments 
          with large language models. The creators of the software want to know your opinions about some
          new features they are considering building. Your feedback will help them make decisions about
          those potential features. """


In [19]:
# Construct questions for the task
q1 = QuestionMultipleChoice(
    question_name="use_often",
    question_text=context
    + """Consider the following new feature: {{ content }}
    How often do you think you would use it?""",
    question_options=["Never", "Occasionally", "Frequently", "All the time"],
)

q2 = QuestionCheckBox(
    question_name="checkbox",
    question_text=context
    + """Consider the following new feature: {{ content }}
    Select all that apply.""",
    question_options=[
        "This feature would be useful to me.",
        "This feature would make me more productive.",
        "This feature will be important to me.",
        "The benefits of this feature are not clear to me.",
        "I would like to see some examples of how to use this feature.",
    ],
)

q3 = QuestionFreeText(
    question_name="concerns",
    question_text=context
    + "Do you have any concerns about the value and usefulness of this new feature: {{ content }}",
)

q4 = QuestionLinearScale(
    question_name="likely_to_use",
    question_text=context
    + """Consider the following new feature: {{ content }}
    On a scale from 1 to 5, how likely are you to use this new feature? 
    (1 = not at all likely, 5 = very likely)""",
    question_options=[1, 2, 3, 4, 5],
    option_labels={1: "Not at all likely", 5: "Very likely"},
)

In [6]:
# Create a survey with the questions
survey = Survey(questions=[q1, q2, q3, q4])

In [7]:
# Create some content for the agent to review
contents = [
    "An optional progress bar that shows how many of your questions have been answered while your survey is running.",
    "A method that lets you quickly check what version of the package you have installed.",
    "A method that lets you include questions and responses as context for new questions.",
]

# Parameterize the questions with the content
scenarios = [Scenario({"content": c}) for c in contents]

In [9]:
agent

Agent(name = """Robin""", traits = {'persona': 'You are a middle-aged mom in Cambridge, Massachusetts. \n        You hope to own a driverless minivan in the near future.\n        You are working on an open source Python package for conducting research with AI.', 'age': 45, 'location': 'US', 'industry': 'information technology', 'company': 'Expected Parrot', 'occupation': 'startup cofounder', 'hobbies': 'kayaking, beach walks'})

In [12]:
survey.to_scenario_list().print()

question_name,question_text,question_options,question_type,min_selections,option_labels,max_selections
use_often,You are answering questions about a software package for conducting surveys and experiments with large language models. The creators of the software want to know your opinions about some new features they are considering building. Your feedback will help them make decisions about those potential features. Consider the following new feature: {{ content }} How often do you think you would use it?,"['Never', 'Occasionally', 'Frequently', 'All the time']",multiple_choice,None,None,None
checkbox,You are answering questions about a software package for conducting surveys and experiments with large language models. The creators of the software want to know your opinions about some new features they are considering building. Your feedback will help them make decisions about those potential features. Consider the following new feature: {{ content }} Select all that apply.,"['This feature would be useful to me.', 'This feature would make me more productive.', 'This feature will be important to me.', 'The benefits of this feature are not clear to me.', 'I would like to see some examples of how to use this feature.']",checkbox,None,None,None
concerns,You are answering questions about a software package for conducting surveys and experiments with large language models. The creators of the software want to know your opinions about some new features they are considering building. Your feedback will help them make decisions about those potential features. Do you have any concerns about the value and usefulness of this new feature: {{ content }},None,free_text,None,None,None
likely_to_use,"You are answering questions about a software package for conducting surveys and experiments with large language models. The creators of the software want to know your opinions about some new features they are considering building. Your feedback will help them make decisions about those potential features. Consider the following new feature: {{ content }} On a scale from 1 to 5, how likely are you to use this new feature? (1 = not at all likely, 5 = very likely)","[1, 2, 3, 4, 5]",linear_scale,None,None,None


In [20]:
# Run the survey and store the results, and show a progress bar
results = survey.by(scenarios).by(agent).run(progress_bar=True)

Output()

In [ ]:
# Show all columns of the Results object
results.columns

['agent.age',
 'agent.agent_instruction',
 'agent.agent_name',
 'agent.company',
 'agent.hobbies',
 'agent.industry',
 'agent.location',
 'agent.occupation',
 'agent.persona',
 'answer.checkbox',
 'answer.concerns',
 'answer.likely_to_use',
 'answer.use_often',
 'comment.checkbox_comment',
 'comment.likely_to_use_comment',
 'comment.use_often_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.checkbox_system_prompt',
 'prompt.checkbox_user_prompt',
 'prompt.concerns_system_prompt',
 'prompt.concerns_user_prompt',
 'prompt.likely_to_use_system_prompt',
 'prompt.likely_to_use_user_prompt',
 'prompt.use_often_system_prompt',
 'prompt.use_often_user_prompt',
 'question_options.checkbox_question_options',
 'question_options.concerns_question_options',
 'question_options.likely_to_use_question_options',
 'question_options.use_ofte

In [ ]:
# Print the responses
results.select(
    "content",
    "use_often",
    "checkbox",
    "concerns",
    "likely_to_use",
).print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ scenario                 ┃ answer       ┃ answer                    ┃ answer                   ┃ answer         ┃
┃ .content                 ┃ .use_often   ┃ .checkbox                 ┃ .concerns                ┃ .likely_to_use ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ A method that lets you   │ Frequently   │ ['This feature would be   │ Having a feature that    │ 5              │
│ quickly check what       │              │ useful to me.', 'This     │ allows users to quickly  │                │
│ version of the package   │              │ feature would make me     │ check the version of the │                │
│ you have installed.      │              │ more productive.', 'This  │ package they have        │                │
│                          │              │ feature will be important │ installed is quite       │                │
│                          │              │ to me.', 'I would like to │ valuable. It's essential │                │
│                          │              │ see some examples of how  │ for troubleshooting,     │                │
│                          │              │ to use this feature.']    │ ensuring compatibility   │                │
│                          │              │                           │ with other software, and │                │
│                          │              │                           │ verifying that you have  │                │
│                          │              │                           │ the latest features and  │                │
│                          │              │                           │ bug fixes. It's a        │                │
│                          │              │                           │ fundamental feature that │                │
│                          │              │                           │ supports good software   │                │
│                          │              │                           │ maintenance practices.   │                │
│                          │              │                           │ My only concern would be │                │
│                          │              │                           │ ensuring that the method │                │
│                          │              │                           │ to check the version is  │                │
│                          │              │                           │ simple and intuitive, so │                │
│                          │              │                           │ users of all experience  │                │
│                          │              │                           │ levels can easily use it │                │
│                          │              │                           │ without confusion.       │                │
├──────────────────────────┼──────────────┼───────────────────────────┼──────────────────────────┼────────────────┤
│ An optional progress bar │ All the time │ ['This feature would be   │ The optional progress    │ 5              │
│ that shows how many of   │              │ useful to me.', 'This     │ bar for tracking         │                │
│ your questions have been │              │ feature would make me     │ answered questions       │                │
│ answered while your      │              │ more productive.', 'I     │ during a survey seems    │                │
│ survey is running.       │              │ would like to see some    │ like a practical and     │                │
│                          │              │ examples of how to use    │ user-friendly feature.   │                │
│                          │              │ this feature.']           │ It would provide         │                │
│                          │              │             

In [ ]:
# Post the notebook on the Coop
from edsl import Coop, Notebook

coop = Coop()

notebook = Notebook(path = "digital_twin.ipynb")

coop.create(notebook, description="Digital Twin", visibility="public") 

{'description': None,
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/dd28e24c-03a4-47dc-b2d5-6f8bc78788ce',
 'uuid': 'dd28e24c-03a4-47dc-b2d5-6f8bc78788ce',
 'version': '0.1.30.dev3',
 'visibility': 'public'}